In [1]:
from __future__ import print_function,division

from sklearn.datasets import make_classification
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings('ignore')

D:\Program Files (x86)\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data, target = make_classification(n_samples=1000,n_features=45,n_informative=12,n_redundant=7,random_state=2018)

定义优化的问题，输入为需要优化的超参数，输出为需要优化的目标值

In [3]:
def svccv(C, gamma):
    val = cross_val_score(estimator=SVC(C=C,gamma=gamma,probability=True,random_state=2018),X=data,y=target,scoring='f1',cv=2).mean()
    return val

In [4]:
def rfccv(n_estimators, min_samples_split, max_features):
    val = cross_val_score(
        RFC(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999),
            random_state=2
        ),
        data, target, 'f1', cv=2
    ).mean()
    return val

In [5]:
if __name__ == "__main__":
    # 定义传入优化函数的内置变量，不会被定义为优化超参数
    gp_params = {"alpha": 1e-5}

    svcBO = BayesianOptimization(svccv,
        {'C': (0.001, 1), 'gamma': (0.0001, 0.1)})
    # 贝叶斯初始化必须包含的点
    svcBO.explore({'C': [0.001, 0.01, 0.1], 'gamma': [0.001, 0.01, 0.1]})
    # 贝叶斯优化的各个参数的上下线
    rfcBO = BayesianOptimization(
        rfccv,
        {'n_estimators': (200, 550),
        'min_samples_split': (2, 25),
        'max_features': (0.3, 0.7)}
    )

    svcBO.maximize(init_points=5,n_iter=10, **gp_params)
    print('-' * 53)
    rfcBO.maximize(init_points=5,n_iter=10, **gp_params)

    print('-' * 53)
    print('Final Results')
    print('SVC: %f' % svcBO.res['max']['max_val'])
    print('RFC: %f' % rfcBO.res['max']['max_val'])

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    1 | 00m00s |    0.00000 |    0.0010 |    0.0010 | 
    2 | 00m00s |    0.00000 |    0.0100 |    0.0100 | 
    3 | 00m00s |    0.00000 |    0.1000 |    0.1000 | 
    4 | 00m00s |    0.71034 |   89.9878 |    0.0636 | 
    5 | 00m00s |    0.89753 |   25.4890 |    0.0331 | 
    6 | 00m00s |    0.41218 |   99.3426 |    0.0848 | 
    7 | 00m00s |    0.35456 |    8.7792 |    0.0892 | 
    8 | 00m00s |    0.75723 |   72.3955 |    0.0591 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    9 | 00m03s |    0.83034 |   45.4679 |    0.0001 | 
   10 | 00m03s |    0.82695 |   34.4431 |    0.0001 | 
   11 | 00m03s |    0.82676 |   57.9144 |    0.0001 | 
   12 | 00m03s |    0.82433 |   27.9552 |    0.0001 | 
   13 | 00m02s |    0.82187 |   19.2769 |    0.0001 | 
   14 | 00m02s |    0.83513 | 